# 1. Global sequence alignments

You are given two sequences AGATTAC and AGTTAC. Assume a match score of 1, a gap penalty of 3 and a substitution score of -1. Using these scores, obtain the global alignment of these two sequences manually in the following two steps:

1. Fill in the entries of the F matrix by applying the recurrence relationship for global alignment to these sequences. Please show the back pointers to the matrix entry/entries that give you the maximal score for any entry.

2. Apply the trace back procedure to obtain an optimal alignment. If there are multiple possible alignments, please show all of them along with their traceback paths.

You may take a picture of your alignments and include the image as submission for homework.  Please type the alignments immediately following.

![](1.png)

# 2. Local sequence alignments

Perform a manual local alignment for the sequences, GAAGAGTATA and AAGCTATACC.   Assume a match score of 1, a gap penalty of 3 and a substitution score of -1.  Complete the following steps:

1. Fill in the entries of the F matrix using the recurrence relationship for the local alignment of these sequences. Show back pointers to matrix entry/entries that give you the maximal score.

2. Apply the trace back procedure to generate a local alignment.

You may take a picture of your alignments and include the image as submission for homework.  Please type the alignments immediately following.

![](2.png)

# 3. Dynamic Programming Implementation

Implement dynamic programming algorithms for global (Needleman-Wunsch) and local (Smith-Waterman) alignment of protein sequences.  The implementation should be on Google App Engine platform and use a web form to submit the alignment calculation.  The form should have the following fields:

* Sequence 1
* Sequence 2
* Select global or local alignment
* Match Score, Mismatch Score, Linear Gap penalty

Once submitted, the solution should show all optimal alignments, the scoring matrix and traceback onscreen.  Type the URL of the application below?

In [1]:
# URL
# https://bnroths-166719.appspot.com

import numpy as np

class Alignment:
	def __init__(self, type='global'):
		self.type = 'global'

	def traceback(self, grid, trace, seq1, seq2):
		opt_align = []
		align1 = ''
		align2 = ''
		i, j = np.unravel_index(grid.argmax(), grid.shape)
		
		while i > 0 and j > 0:
			next_step = str(trace[i][j])
			if 'up' in next_step:
				align1 = seq1[i-1] + align1
				align2 = '_' + align2
				i -= 1
			elif 'left' in next_step:
				align1 = '_' + align1
				align2 = seq2[j-1] + align2
				j -= 1
			elif 'diag' in next_step:
				align1 = seq1[i-1] + align1
				align2 = seq2[j-1] + align2
				i -= 1
				j -= 1
		
		opt_align.append(align1)
		opt_align.append(align2)
		
		return opt_align
	def alignment(self, sequence_1, sequence_2, alignment_type='global', match_score=1, mismatch_score=-1, linear_gap=-3):
		if alignment_type not in ('global', 'local'):
			return False
		seq_1_len = len(sequence_1) + 1
		seq_2_len = len(sequence_2) + 1
		
		grid = np.zeros((seq_1_len, seq_2_len))
		trace = np.zeros((seq_1_len, seq_2_len)).tolist()
		
		# init matrix with edges gap penelty
		if alignment_type == 'global':
			for i in range(seq_1_len):
				grid[i][0] = linear_gap * i
			for j in range(seq_2_len):
				grid[0][j] = linear_gap * j
		elif alignment_type == 'local':
			for i in range(seq_1_len):
				grid[i][0] = 0
			for j in range(seq_2_len):
				grid[0][j] = 0
				
		print grid
		

		for i in range(1, seq_1_len):
			for j in range(1, seq_2_len):

				## does diag_match?
				if sequence_1[i-1] == sequence_2[j-1]:
					match_boost = match_score
				else:
					match_boost = mismatch_score

				if alignment_type == 'global':
					up   = grid[i-1][j] + linear_gap
					left = grid[i][j-1] + linear_gap
					diag = grid[i-1][j-1] + match_boost
					grid[i][j] = max(up, left, diag)
					trace_pointer = ''
					if up == grid[i][j]:
						trace_pointer += 'up'
					if left == grid[i][j]:
						trace_pointer += 'left'
					if diag == grid[i][j]:
						trace_pointer += 'diag'
					trace[i][j] = str(trace_pointer)
				elif alignment_type == 'local':
					diag = grid[i-1][j-1] + match_boost
					grid[i][j] = max(diag, 0)
					if diag > 0:
						trace[i][j] = 'diag'
		alignment = self.traceback(grid, trace, sequence_1, sequence_2)
		# print alignment
		return grid, trace, alignment, int(grid.max())


In [2]:
a = Alignment()
grid, trace, alignment, score = a.alignment('AGATTAC', 'AGTTAC', alignment_type='global')
#a.alignment('AAGCTATACC', 'GAAGAGTATA', alignment_type='local')
print grid
print alignment
print score

[[  0.  -3.  -6.  -9. -12. -15. -18.]
 [ -3.   0.   0.   0.   0.   0.   0.]
 [ -6.   0.   0.   0.   0.   0.   0.]
 [ -9.   0.   0.   0.   0.   0.   0.]
 [-12.   0.   0.   0.   0.   0.   0.]
 [-15.   0.   0.   0.   0.   0.   0.]
 [-18.   0.   0.   0.   0.   0.   0.]
 [-21.   0.   0.   0.   0.   0.   0.]]
[[  0.  -3.  -6.  -9. -12. -15. -18.]
 [ -3.   1.  -2.  -5.  -8. -11. -14.]
 [ -6.  -2.   2.  -1.  -4.  -7. -10.]
 [ -9.  -5.  -1.   1.  -2.  -3.  -6.]
 [-12.  -8.  -4.   0.   2.  -1.  -4.]
 [-15. -11.  -7.  -3.   1.   1.  -2.]
 [-18. -14. -10.  -6.  -2.   2.   0.]
 [-21. -17. -13.  -9.  -5.  -1.   3.]]
['AGATTAC', 'AG_TTAC']
3


# 4.  Statisics of Pairwise Alignments

Perform sequence alignment of the following sequences:

```
PAWHEAE
HEAGAWGHEE
```
Assume a match score of 1, a gap penalty of 3 and a substitution score of -1.  What is the alignment score?

In [3]:
grid, trace, alignment, score = a.alignment('PAWHEAE', 'HEAGAWGHEE', alignment_type='local')
print score
print alignment

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
3
['HEA', 'HEA']



Next, generate 100 random sequences with the same amino acid distribution as `PAWHEAE` and perform an alignment of each sequence to `HEAGAWGHEE`.   Calculate the Z-score.  What does the Z-score say about the significance of the alignment?   Are the scores normally distributed?  *You can use SciPy or any other modules to help.*

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

%matplotlib inline


from random import shuffle
a = Alignment()

def run_n_times(n=100):
    sequence_1 = 'PAWHEAE'
    scores = []
    for i in range(n):
        # print i, sequence_1
        sequence_1 = list(sequence_1)
        shuffle(sequence_1)
        sequence_1 = "".join(sequence_1)
        grid, trace, alignment, score = a.alignment(sequence_1, 'HEAGAWGHEE', alignment_type='local')
        scores.append(score)

    plt.hist(scores)

    plt.xlabel('Score')
    plt.ylabel('Count')
    plt.title(r'Scores Histogram n=%s' % n)
    plt.xlim([0, 5])
    # Tweak spacing to prevent clipping of ylabel
    fig.tight_layout()
    plt.show()
    
    z = 1.0*(np.max(scores) - np.mean(scores))/np.std(scores)
    #print np.mean(scores), np.std(scores), np.max(scores), z
    return np.mean(scores), np.std(scores), np.max(scores), z, scores

mean, std, max_score, z, scores = run_n_times(100)


z, pval = stats.normaltest(scores)

if(pval < 0.05):
    print "Not normal distribution"
else:
    print "Normal distribution"
    
print "Z score = %s" % z

Repeat the process above, but now generate 1,000 and 10,000 random sequences and calculate the Z-score for each set.  Does the change in number of sequences alter your evaluation of the evolutionary relatedness of the sequences?

In [ ]:
mean, std, max_score, z, scores = run_n_times(1000)

z, pval = stats.normaltest(scores)

if(pval < 0.05):
    print "Not normal distribution"
else:
    print "Normal distribution"
    
print "Z score = %s" % z

In [ ]:
mean, std, max_score, z, scores = run_n_times(10000)

z, pval = stats.normaltest(scores)

if(pval < 0.05):
    print "Not normal distribution"
else:
    print "Normal distribution"
    
print "Z score = %s" % z

Considering the wall time for each run, comment of the feasibility for computing the Z-score significance while searching a large database of millions of sequences.

Though I ran the script for only a 10,000 iterations it did not take super long (finished in less than 5 seconds)

# 5.  Recent Approaches in Sequence Alignemnt

Search PubMed and identify a paper on a different sequence alignment method.  Do not use BLAST or FASTA.  Briefly (~1 paragraph) discuss what is different in these approaches and how they improve on Needleman-Wunch and/or Smith-Waterman.  You answer does not have to be uneccesarily technical, but please provide a concrete example of the improvement it makes.